In [1]:
import sys
print(sys.path)
%run ./data.ipynb
%run ./model.ipynb
%run ./loss.ipynb
%run ./utils.ipynb




['D:\\deeplearning_project\\retina vessel segmentation', 'C:\\Users\\krish\\anaconda3\\python39.zip', 'C:\\Users\\krish\\anaconda3\\DLLs', 'C:\\Users\\krish\\anaconda3\\lib', 'C:\\Users\\krish\\anaconda3', '', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\krish\\.ipython']
['D:\\deeplearning_project\\retina vessel segmentation', 'C:\\Users\\krish\\anaconda3\\python39.zip', 'C:\\Users\\krish\\anaconda3\\DLLs', 'C:\\Users\\krish\\anaconda3\\lib', 'C:\\Users\\krish\\anaconda3', '', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg', 'C:\\Users\\krish\\anaconda3\\lib\\site-pac

In [4]:
import os
import time
from glob import glob

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision.transforms import ToTensor

# # from data import DriveDataset
# from model import attention_unet
# from loss import DiceLoss, DiceBCELoss
# from utils import seeding, create_dir, epoch_time




In [5]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0

    model.train()
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(loader)

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

    return epoch_loss / len(loader)


In [ ]:
# Seeding
seeding(42)

# Directories
create_dir("files")

# Load dataset
train_x = sorted(glob("./Data1/train/image/*"))
train_y = sorted(glob("./Data1/train/mask/*"))
valid_x = sorted(glob("./Data1/test/image/*"))
valid_y = sorted(glob("./Data1/test/mask/*"))

data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
print(data_str)

# Hyperparameters
H = 512
W = 512
size = (H, W)
batch_size = 2
num_epochs = 10
lr = 1e-4
checkpoint_path = "files/checkpoint.pth"

# Dataset and loader
train_dataset = DriveDataset(train_x, train_y)
valid_dataset = DriveDataset(valid_x, valid_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = attention_unet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = DiceBCELoss()  # Use an appropriate loss function for your task

# Training the model
best_valid_loss = float("inf")
for epoch in range(num_epochs):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    valid_loss = evaluate(model, valid_loader, loss_fn, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), checkpoint_path)

    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

    data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs:.2f}s\n'
    data_str += f'\tTrain Loss: {train_loss:.3f}\n'
    data_str += f'\tVal. Loss: {valid_loss:.3f}\n'
    print(data_str)
